**Part 1: Acquisition & Integration**
1. Finalize required ACS variables and write the correct Python API script. 
2. Load and profile the local unemployment CSV. 
3. Clean, standardize, and merge both datasets on State/FIPS code and Year.

<sup>This is also slightly more involved because in the ACS variables, although the data is there, the codes used might vary year by year. Since I plan to view over a longer period of time I will need to make sure I have all of the correct codes needed.</sup>

**Part 2: Profiling & Cleaning**
1. Address missing values, outliers, and data type inconsistencies. 
2. Create derived metrics as needed.
3. ~~Conduct initial data profiling.~~

<sup>This part is just initially checking to make sure that the data makes sense (such as 100% being unemployed is probably bad) ~~and setting up for any analysis that will be done later.~~</sup>

Above are copied sections of my [ProjectPlan.md](../ProjectPlanItems/ProjectPlan.md) file. I will be working with both of the different datasets here. I will do everything except for data profliling.

### Range of Years
As shown in [DatasetsExamples.ipynb](../ProjectPlanItems/DatasetsExamples.ipynb), to work with the census API I will need to get all the codes from each year. The range of years is different for both of the datasets, but they have a large overlap.

For "American Community Survey 1-Year Data (2005-2024)" it is given in the name that it is for 2005-2024 and the code below shows the year range for the "Unemployment in America Per US State" data.

In [ ]:
import pandas as pd

df_unemploy = pd.read_csv("../Unemployment_in_America_Per_US_State.csv")
range_years = (df_unemploy['Year'].min(), df_unemploy['Year'].max())
print(f"Range of Years: {range_years[0]}-{range_years[1]}")

Range of Years: 1976-2022


Based on this we have a common range of: 2005-2022*.

I will need to get all the codes from the ACS for every year and then do a simple filter on `df_unemploy`.

<sup>*The year 2020 will be removed. Explanation within Dataset 1 section below.</sup>

### Dataset 1: ACS
In [DatasetsExamples.ipynb](../ProjectPlanItems/DatasetsExamples.ipynb), I have already gathered the codes for the year 2022 manually by looking at the table for that year. We can see the variable codes from using a website formated like this: 

`https://api.census.gov/data/*YEAR*/acs/acs1/variables.html`

Where you change the `*YEAR*` with the actual number. This can get tedious, so to speed it up we can do a sort of loop to get all the variables.


**One thing to note before the loop**, is that within these variables they are sectioned by tables. Each variable starts with the table ID which is why in my example from before you saw code like this in the `VARIABLE_CODES` list:

`'B15003_001E',  # Total Population 25 Years and Over`

The first part of that code (before the underscore) is the table ID which then is followed by the varaible ID (after the underscore). So using our example from about we have a table ID of B15003 and a variable ID of 001E.

The variable codes are actually consistent throughout the years, **BUT** the table ID is not. It is somewhat consistent with needing to follow this format:
- `B15003` for 2009-2019 and 2021-2022
- `B15002` for 2005-2008

The table codes have a major difference in that while they have the same data, the `B15002` is split between male and female variable codes. In those years I will just combine them to achieve the same effect of `B15003` which is combined already*. This doesn't change the looping functionality too much as all I will be doing is just checking the year and using the appropriate codes.

You may have noticed that 2020 was not included above. This is because of the pandemic. On their [website](https://www.census.gov/programs-surveys/acs/technical-documentation/table-and-geography-changes/2020/1-year.html) they say, "\[t]he Census Bureau did not release its standard 2020 ACS 1-year estimates because of the impacts of the COVID-19 pandemic. Instead, the Census Bureau released experimental estimates from the 1-year data.  They are posted on the 2020 ACS 1-Year Experimental Data Release page. They will not be released on data.census.gov." While I could use this experimental data, I had to download it as an xlsx file shown here:
![2020Image](Data_2020image.png)

With the amount of data we have, I am choosing to remove this year. I could choose to use acs5 instead of acs1, but I do not want to change it.

I will also be using one other library: `time`. This is for a simple sleep command so that with all this looping I do not overload the API. The format of the code will look similar to the examples file with the added changes stated above.

<sup>*Every year does have the `B15002` table, but to save time I am choosing not to use it for the other years since `B15003` is already made.</sup>

***Note: the code box below takes about 50-60 seconds to run. You can skip this I have already ran it and created a csv file.***

In [140]:
# needed imports
import pandas as pd
from census import Census
import time

# api key
with open("../census_apikey.txt") as f:
    API_KEY = f.read().strip()

# set up census info
c = Census(API_KEY)
YEARS = [y for y in range(2005, 2023) if y != 2020] 
GEOGRAPHY_FOR = 'state:*' 
SURVEY_METHOD = c.acs1

# this similar to VARIABLE_CODES in examples file
# note it has both B15003 and B15002 tables along with their varaible codes after
# formated as (combined, male, female) from their tables
VARIABLE_MAPPING = {
    # there is a total variable without the male/female split:
    'Total_Pop_25_Plus':            ('B15003_001E', 'B15002_001E', None),
    'Edu_12th_No_Diploma':          ('B15003_016E', 'B15002_010E', 'B15002_027E'),
    # before 2009 they had this included with "Edu_GED_or_Equiv":
    'Edu_Regular_HS_Diploma':       ('B15003_017E',  None,         None),
    # this is the "Edu_GED_or_Equiv" but for renaming purposes changed here:
    'Edu_HS_GED_or_Equiv':          ('B15003_018E', 'B15002_011E', 'B15002_028E'), 
    'Edu_Some_College_Less_1_Year': ('B15003_019E', 'B15002_012E', 'B15002_029E'),
    'Edu_Some_College_1_Plus_Year': ('B15003_020E', 'B15002_013E', 'B15002_030E'),
    'Edu_Associates_Degree':        ('B15003_021E', 'B15002_014E', 'B15002_031E'),
    'Edu_Bachelors_Degree':         ('B15003_022E', 'B15002_015E', 'B15002_032E'),
    'Edu_Masters_Degree':           ('B15003_023E', 'B15002_016E', 'B15002_033E'),
    'Edu_Professional_Degree':      ('B15003_024E', 'B15002_017E', 'B15002_034E'),
    'Edu_Doctorate_Degree':         ('B15003_025E', 'B15002_018E', 'B15002_035E'),
}

# just getting the codes themselves from the mapping instead of retyping
ALL_RAW_CODES = [code for codes in VARIABLE_MAPPING.values() for code in codes if code is not None]
ALL_RAW_CODES = list(set(ALL_RAW_CODES))
ALL_RAW_CODES.insert(0, 'NAME')

# start looping through the years and collecting the data
all_data = []
for year in YEARS:
    # first we need to get the codes that will be required for each year
    required_codes = ['NAME']

    # if we need to use the other table as explained above
    is_b15002 = False

    if year < 2009:
        is_b15002 = True
        for _, (_, code_m, code_f) in VARIABLE_MAPPING.items():
            if code_m: required_codes.append(code_m)
            if code_f: required_codes.append(code_f)
    else:
        for _, (code_b15003, _, _) in VARIABLE_MAPPING.items():
            required_codes.append(code_b15003)

    # in a try except as I need to debug code
    try:
        # actual api call
        data_raw = SURVEY_METHOD.get(
            list(set(required_codes)), # this just makes sure codes are in a unique list
            {'for': GEOGRAPHY_FOR},
            year = year
        )
        df = pd.DataFrame(data_raw)
        df['Year'] = year

        # the names are the codes, so we need to do some slight cleaning
        df_final = df[['NAME', 'state', 'Year']].copy()

        # get the correct names based on table
        for clean_name, (code_b15003, code_b15002_m, code_b15002_f) in VARIABLE_MAPPING.items():
            # for B15002 we need to add female and male
            if is_b15002:
                if code_b15002_m is None: continue

                if clean_name == 'Total_Pop_25_Plus':
                    # however, this total is not split
                    df_final[clean_name] = df[code_b15002_m]
                else:
                    # Sum of male + female components (make sure they are numbers when adding)
                    male_val = pd.to_numeric(df[code_b15002_m])
                    female_val = pd.to_numeric(df[code_b15002_f])
                    df_final[clean_name] = male_val + female_val
            else: 
                # this is easier for B15003
                df_final[clean_name] = df[code_b15003]
        
        if not is_b15002:
            # explained in code above, but combining so it stays consistent and removing extra
            df_final['Edu_HS_GED_or_Equiv'] = df_final['Edu_HS_GED_or_Equiv'] + df_final['Edu_Regular_HS_Diploma']
            df_final = df_final.drop(columns=['Edu_Regular_HS_Diploma'])

        all_data.append(df_final)

    except Exception as e:
        # catch errors if any
        print(f"--- FAILED to retrieve data for year {year}. Error: {e} ---")

    time.sleep(0.5) # API courtesy pause

# combine the results
df_education = pd.concat(all_data, ignore_index = True)

# same code/math as examples file
df_education['HS_Grad_or_Higher_Count'] = (
    df_education['Edu_HS_GED_or_Equiv'] +
    df_education['Edu_Some_College_Less_1_Year'] +
    df_education['Edu_Some_College_1_Plus_Year'] +
    df_education['Edu_Associates_Degree'] +
    df_education['Edu_Bachelors_Degree'] +
    df_education['Edu_Masters_Degree'] +
    df_education['Edu_Professional_Degree'] +
    df_education['Edu_Doctorate_Degree']
)

for col in df_education.columns.tolist():
    if col != 'NAME':
        df_education[col] = df_education[col].astype(int)

df_education['HS_Grad_or_Higher_Pct'] = (
    df_education['HS_Grad_or_Higher_Count'] / df_education['Total_Pop_25_Plus']
) * 100
df_education['HS_Grad_or_Higher_Pct'] = round(df_education['HS_Grad_or_Higher_Pct'], 2)

df_education = df_education.rename(columns = {'NAME': 'State', 'state': 'State_FIPS'})

# finally write out a csv file for easy access later and no need to rerun code here
output_filename = '../ACS (2005-2019, 2021-2022).csv'
df_education.to_csv(output_filename, index = False)

Now time to just take a quick look. Not doing analysis here, just checking to make sure data was transfered correctly.

In [141]:
import pandas as pd # in case above section wasn't ran
df_education = pd.read_csv("../ACS (2005-2019, 2021-2022).csv")
df_education.sample(n = 5)

,State,State_FIPS,Year,Total_Pop_25_Plus,Edu_12th_No_Diploma,Edu_HS_GED_or_Equiv,Edu_Some_College_Less_1_Year,Edu_Some_College_1_Plus_Year,Edu_Associates_Degree,Edu_Bachelors_Degree,Edu_Masters_Degree,Edu_Professional_Degree,Edu_Doctorate_Degree,HS_Grad_or_Higher_Count,HS_Grad_or_Higher_Pct
798,Louisiana,22,2021,3117186,59256,1023442,178427,464414,211237,523475,205307,58391,37196,2701889,86.68
518,Wyoming,56,2014,388962,4395,106759,35128,73289,41611,69557,23807,5370,4599,360120,92.58
139,Ohio,39,2007,7627615,142575,2805856,493399,956671,548843,1161216,479887,124997,69204,6640073,87.05
612,Hawaii,15,2016,990937,11536,277743,53877,158800,104261,211850,67803,21989,14951,911274,91.96
422,New Jersey,34,2013,6086705,104427,1751336,304286,715243,390906,1374616,615250,146050,89953,5387640,88.51


Quick check on Illinois from another source:
- https://fred.stlouisfed.org/series/GCT1501IL
- csv from link downloaded with year by year data: [GCT1501IL.csv](GCT1501IL.csv)
- this has 2006-2024, so modifying code slighty to check those years

In [142]:
remove = ("2024-01-01", "2023-01-01", "2020-01-01")
df_fred = pd.read_csv("GCT1501IL.csv")
df_fred = df_fred[~df_fred['observation_date'].isin(remove)].reset_index() # removes 2020, 2023, 2024

df_il = df_education[(df_education['State'] == 'Illinois')][['Year', 'HS_Grad_or_Higher_Pct']]

df_check = round(df_il[df_il['Year'] != 2005], 1).reset_index()
df_check['GCT1501IL'] = df_fred['GCT1501IL']
df_check['Difference'] = df_check['HS_Grad_or_Higher_Pct'] - df_check['GCT1501IL']
print(f'Max difference: {df_check['Difference'].max()}%')
print(f'Min difference: {df_check['Difference'].min()}%')
print(f'Mean difference: {df_check['Difference'].mean()}%')
print(f'Median difference: {df_check['Difference'].median()}%')
df_check

Max difference: 0.0%
Min difference: 0.0%
Mean difference: 0.0%
Median difference: 0.0%


,index,Year,HS_Grad_or_Higher_Pct,GCT1501IL,Difference
0,65,2006,85.0,85.0,0.0
1,117,2007,85.7,85.7,0.0
2,169,2008,85.9,85.9,0.0
3,221,2009,86.4,86.4,0.0
4,273,2010,86.9,86.9,0.0
5,325,2011,87.2,87.2,0.0
6,377,2012,87.6,87.6,0.0
7,457,2013,87.8,87.8,0.0
8,481,2014,88.2,88.2,0.0
9,561,2015,88.6,88.6,0.0


I, at first, was not looking for another dataset to compare to, but when I was looking up for a specific year on google this came up and it was perfect. The year 2005 is not checked based on the other source, but based on everything else I can only assume that it would be correct. I also rounded my percentages because as that was how theirs were. They cite using the same data from the census confirming that my code above ended up working.

<sup><sup>**Sidenote:** I was pretty sure that my code with the API was right, but I did not expect it to be basically perfect (unless we are both wrong). This actually made me pretty excited. I wrote the API code above and this code at seperate times. I did not know this specific data was already made, so technically I did not need to do all the Census API myself, but I think it was was very useful for me to debug as well as get extra info potentially.</sup></sup>

### Dataset 2: Unemployment

This data is much simpler as it is a csv that I have downloaded.

The first thing is to make sure the years will match with my Dataset 1 (ACS) above.

In [143]:
df_unemploy = pd.read_csv("../Unemployment in America Per US State.csv")
YEARS = [y for y in range(2005, 2023) if y != 2020]
df_unemploy = df_unemploy[df_unemploy['Year'].isin(YEARS)].reset_index()
df_unemploy.sample(n = 5)

,index,FIPS Code,State/Area,Year,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area
350,18794,35,New Mexico,2005,7,"1,436,873","907,701",63.2,"861,045",59.9,"46,656",5.1
2927,21371,15,Hawaii,2009,8,"982,384","631,317",64.3,"588,153",59.9,"43,164",6.8
5028,23472,49,Utah,2012,11,"2,040,092","1,364,818",66.9,"1,301,946",63.8,"62,872",4.6
6278,24722,27,Minnesota,2014,11,"4,265,112","2,991,030",70.1,"2,872,957",67.4,"118,073",3.9
418,18862,50,Vermont,2005,8,"495,840","350,538",70.7,"338,360",68.2,"12,178",3.5


Next because this dataset is seperated by months as well, I want this to be consolidated into only years. I will do this by grouping by State and Year and getting the averages. Note above values where some of them have `,`s in the number which means it is a string that needs to be converted.

In [144]:
group_cols = ['FIPS Code', 'State/Area', 'Year']

avg_cols = [
    'Total Civilian Non-Institutional Population in State/Area',
    'Total Civilian Labor Force in State/Area',
    'Total Employment in State/Area',
    'Total Unemployment in State/Area'
]

for col in df_unemploy.columns.tolist():
    if col not in group_cols:
        df_unemploy[col] = df_unemploy[col].astype(str).str.replace(',', '')
        df_unemploy[col] = df_unemploy[col].astype(float)

df_unemploy_yearly = df_unemploy.groupby(group_cols)[avg_cols].mean().reset_index()

# add "Avg Annual" to the column names
df_unemploy_yearly.rename(columns={col: 'Avg Annual ' + col for col in avg_cols}, inplace = True)

# new column names
avg_unemploy = 'Avg Annual Total Unemployment in State/Area'
avg_labor_force = 'Avg Annual Total Civilian Labor Force in State/Area'
avg_pop = 'Avg Annual Total Civilian Non-Institutional Population in State/Area'
avg_employment = 'Avg Annual Total Employment in State/Area'


# 1. get the yearly unemployment rate
df_unemploy_yearly['Annual Percent (%) of Unemployed in State/Area'] = round((
    df_unemploy_yearly[avg_unemploy] /
    df_unemploy_yearly[avg_labor_force]
) * 100, 2)

# 2. get the yearly labor force participation rate
df_unemploy_yearly['Annual Percent (%) of Labor Force Population'] = round((
    df_unemploy_yearly[avg_labor_force] /
    df_unemploy_yearly[avg_pop]
) * 100, 2)

# 3. get the yearly employment population ratio
df_unemploy_yearly['Annual Percent (%) of Total Employment in State/Area'] = round((
    df_unemploy_yearly[avg_employment] /
    df_unemploy_yearly[avg_pop]
) * 100, 2)

# round to nearest int for averages because you can't have a fraction of a person
make_int = ['FIPS Code', 'Avg Annual Total Civilian Non-Institutional Population in State/Area', 
            'Avg Annual Total Civilian Labor Force in State/Area', 'Avg Annual Total Employment in State/Area', 
            'Avg Annual Total Unemployment in State/Area']
for col in make_int:
    df_unemploy_yearly[col] = round(df_unemploy_yearly[col]).astype(int)

df_unemploy_yearly.sample(n = 5)

,FIPS Code,State/Area,Year,Avg Annual Total Civilian Non-Institutional Population in State/Area,Avg Annual Total Civilian Labor Force in State/Area,Avg Annual Total Employment in State/Area,Avg Annual Total Unemployment in State/Area,Annual Percent (%) of Unemployed in State/Area,Annual Percent (%) of Labor Force Population,Annual Percent (%) of Total Employment in State/Area
255,19,Iowa,2005,2275361,1628542,1558853,69689,4.28,71.57,68.51
847,54,West Virginia,2019,1452860,795462,756184,39278,4.94,54.75,52.05
217,16,Idaho,2018,1346386,862480,837708,24772,2.87,64.06,62.22
864,55,Wisconsin,2019,4670190,3111800,3012775,99025,3.18,66.63,64.51
89,8,Colorado,2009,3838336,2731197,2516161,215036,7.87,71.16,65.55


### Final Checks

First we need to do some simple checks before merging them.

Both should have the same `Year`s and `State`s* within.

<sup>*It says state, but both datasets should have American territories.</sup>

In [145]:
ed_states = set(df_education['State'])
unemploy_states = set(df_unemploy_yearly['State/Area'])

print('In df_unemploy_yearly and not df_education:')
diff1 = list(unemploy_states - ed_states)
print(diff1)

print('\nIn df_education and not df_unemploy_yearly:')
diff2 = list(ed_states - unemploy_states)
print(diff2)

print('\nOverall change to remove:')
totaldifff = diff1 + diff2
print(totaldifff)

print('\nYear check:')
ed_year = df_education['Year'].unique()
unemploy_year = df_unemploy_yearly['Year'].unique()
print(set(ed_year) == set(unemploy_year))

In df_unemploy_yearly and not df_education:
['Los Angeles County', 'New York city']

In df_education and not df_unemploy_yearly:
['Puerto Rico']

Overall change to remove:
['Los Angeles County', 'New York city', 'Puerto Rico']

Year check:
True


Firstly, the Years are good so no worries there. I can remove "Puerto Rico" from `df_education` pretty easily. For both of the cities in `df_unemploy`, I double checked with the [Kaggle website](https://www.kaggle.com/datasets/justin2028/unemployment-in-america-per-us-state?select=Unemployment+in+America+Per+US+State.csv) where I got the data and saw that "three other areas are also being tracked in order to increase the analytical potential of the dataset." This tells me that these are seperate from the states they belong to so I can remove them as well. 

In [146]:
common_states = ed_states.intersection(unemploy_states)

df_education = df_education[df_education['State'].isin(common_states)].reset_index(drop=True)
df_unemploy_yearly = df_unemploy_yearly[df_unemploy_yearly['State/Area'].isin(common_states)].reset_index(drop=True)
print(set(df_education['State']) == set(df_unemploy_yearly['State/Area']))

True


Now quick check on the sizes as they should be the same:

In [147]:
print(f'Rows in df_education: {len(df_education)}')
print(f'Rows in df_unemploy_yearly: {len(df_unemploy_yearly)}')

Rows in df_education: 867
Rows in df_unemploy_yearly: 867


Final Look at them both individually:

In [148]:
df_unemploy_yearly

,FIPS Code,State/Area,Year,Avg Annual Total Civilian Non-Institutional Population in State/Area,Avg Annual Total Civilian Labor Force in State/Area,Avg Annual Total Employment in State/Area,Avg Annual Total Unemployment in State/Area,Annual Percent (%) of Unemployed in State/Area,Annual Percent (%) of Labor Force Population,Annual Percent (%) of Total Employment in State/Area
0,1,Alabama,2005,3477391,2140116,2044937,95179,4.45,61.54,58.81
1,1,Alabama,2006,3522273,2169933,2083033,86900,4.00,61.61,59.14
2,1,Alabama,2007,3557957,2180868,2092450,88418,4.05,61.30,58.81
3,1,Alabama,2008,3591892,2177201,2049357,127844,5.87,60.61,57.06
4,1,Alabama,2009,3621410,2157016,1939467,217549,10.09,59.56,53.56
...,...,...,...,...,...,...,...,...,...,...
862,56,Wyoming,2017,445796,293841,281120,12722,4.33,65.91,63.06
863,56,Wyoming,2018,445661,292595,280732,11862,4.05,65.65,62.99
864,56,Wyoming,2019,447440,293375,282455,10920,3.72,65.57,63.13
865,56,Wyoming,2021,454147,289154,275786,13368,4.62,63.67,60.73


In [149]:
df_education

,State,State_FIPS,Year,Total_Pop_25_Plus,Edu_12th_No_Diploma,Edu_HS_GED_or_Equiv,Edu_Some_College_Less_1_Year,Edu_Some_College_1_Plus_Year,Edu_Associates_Degree,Edu_Bachelors_Degree,Edu_Masters_Degree,Edu_Professional_Degree,Edu_Doctorate_Degree,HS_Grad_or_Higher_Count,HS_Grad_or_Higher_Pct
0,Alabama,1,2005,2944634,72646,941983,175947,418334,196856,398907,164897,43349,23455,2363728,80.27
1,Alaska,2,2005,387609,4110,112102,28929,76352,29751,66674,28723,6375,3969,352875,91.04
2,Arizona,4,2005,3703355,86885,948896,290439,618743,300033,601547,241259,68500,35406,3104823,83.84
3,Arkansas,5,2005,1777242,43257,631853,119791,254394,98353,223069,78480,21829,11939,1439708,81.01
4,California,6,2005,22299041,675434,4868430,1435974,3256900,1709579,4215208,1569864,505644,295244,17856843,80.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Virginia,51,2022,5978782,88806,1431624,360707,706905,451737,1409942,818944,165133,128015,5473007,91.54
863,Washington,53,2022,5470209,95516,1176424,398743,762268,544596,1302707,624627,129492,102857,5041714,92.17
864,West Virginia,54,2022,1262566,18958,484166,81446,135471,111246,188793,88210,21126,14957,1125415,89.14
865,Wisconsin,55,2022,4088500,65281,1209135,270280,528285,455882,885548,344375,72109,56389,3822003,93.48


### Merging

Now that the data is all prepared, I can merge them together into one dataset. I am wanting to merge by `State`, `State_FIPS`, and `Year`.

In [152]:
# first make sure the names are the same in both
df_unemploy_yearly.rename(columns = {'State/Area': 'State', 'FIPS Code': 'State_FIPS'}, inplace = True)

merge_keys = ['State', 'State_FIPS', 'Year']

df_merged = pd.merge(
    df_education,
    df_unemploy_yearly,
    on = merge_keys,
    how = 'inner'
)

output_filename = '../merged_education_unemployment_data.csv'
df_merged.to_csv(output_filename, index = False)

df_merged.sample(n = 5)

,State,State_FIPS,Year,Total_Pop_25_Plus,Edu_12th_No_Diploma,Edu_HS_GED_or_Equiv,Edu_Some_College_Less_1_Year,Edu_Some_College_1_Plus_Year,Edu_Associates_Degree,Edu_Bachelors_Degree,...,Edu_Doctorate_Degree,HS_Grad_or_Higher_Count,HS_Grad_or_Higher_Pct,Avg Annual Total Civilian Non-Institutional Population in State/Area,Avg Annual Total Civilian Labor Force in State/Area,Avg Annual Total Employment in State/Area,Avg Annual Total Unemployment in State/Area,Annual Percent (%) of Unemployed in State/Area,Annual Percent (%) of Labor Force Population,Annual Percent (%) of Total Employment in State/Area
318,Idaho,16,2011,998460,15521,275033,82382,186302,88949,174646,...,9487,884761,88.61,1179092,764067,701596,62471,8.18,64.80,59.50
150,West Virginia,54,2007,1261455,20328,517789,71653,139370,76885,135597,...,6883,1024092,81.18,1444435,804399,765698,38701,4.81,55.69,53.01
485,Montana,30,2014,695850,8505,213165,45863,124370,57211,135401,...,8545,644448,92.61,806981,512845,488949,23895,4.66,63.55,60.59
549,Hawaii,15,2015,985914,12292,267568,55400,157806,105951,205033,...,13213,895919,90.87,1086642,670625,647696,22930,3.42,61.72,59.61
316,Georgia,13,2011,6308961,101783,1849141,341468,970101,419954,1119648,...,69822,5319629,84.32,7381036,4739138,4258202,480936,10.15,64.21,57.69


### Done

As a summary of this document:
- Extract (API/CSV)
- Transform (Clean strings, aggregate averages, normalize columns)
- Validate (Compare vs. FRED)
- Derived Metrics (As needed in both)
- Load (Merge and Save)

The resulting [merged_education_unemployment_data.csv](../merged_education_unemployment_data.csv) being made.

<sup>This file is not for analysis. Just to set up and merge the databases for later use. **This file is done.**</sup>